<a href="https://colab.research.google.com/github/wangwangwang77/Master_thesis/blob/main/master_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import math
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import matplotlib.patches as mpatch
from scipy.stats import norm
import scipy.stats as ss
from functools import reduce

##Heston Model
The Heston Model is a stochastic volatility model whose time evolution is given by the following system of stochastic differential equations:
$$dS_{t}=rS_{t}dt+\sigma_{t}S_{t}dZ_{t} $$ 
$$d\sigma_{t}^{2}=\kappa(\theta-\sigma_{t}^{2})dt+\nu\sqrt{\sigma_{t}^{2}}dB_{t}$$
$\rho\neq0$ denotes the correlation parameter between Brownian motion $Z$ and $B$. In our simulation, we assume $r=0$. 
We rewrite $Z_{t}$ as:
$$Z_{t}=\sqrt{1-\rho^{2}}B_{t}+\rho W_{t}$$
where $B_{t}$ and $Z_{t}$ are independent brownian motion and the Heston Model can be written as:
$$dS_{t}=S_{t}\sigma_{t}(\sqrt{1-\rho^{2}}B_{t}+\rho W_{t})$$

#Crude Monte Carlo
We implement discretization schemes for both the stock and the volatility. Thess dynamics looks as follows:
\begin{align*}
\sigma_{i}^{2}&=\kappa(\theta-\sigma_{i}^{2})\Delta t+\xi\sqrt{\sigma_{i-1}^{2}}\Delta W\\
S_{i}&=S_{i-1}+\sqrt{\sigma_{i-1}^{2}}S_{i-1}(\sqrt{1-\rho^{2}}\Delta B+\rho\Delta W)
\end{align*}
We want to find the call option price:
$$E[\max(S_{T}-K)]$$






In [63]:
from scipy.stats import norm
def cmcOptionPrice(T, m, F0, K,sigma_0,rho= -0.5, gamma=0.3, theta=0.09, ka=1,n=5000,r=0):
    delta = T/n
    deltaW1 = np.random.normal(0,np.sqrt(delta),(m,n))
    deltaW2 = np.random.normal(0,np.sqrt(delta),(m,n))
    V = np.zeros((m,n+1))
    V[:,0] = sigma_0
    S = np.zeros((m,n+1))
    S[:,0] = F0
    for i in range(n):
       V[:,i+1] = np.maximum(V[:,i]+ka*(theta-V[:,i])*delta+gamma*np.sqrt(V[:,i])*deltaW2[:,i],0)
       S[:,i+1] = S[:,i]+ np.sqrt(V[:,i])*S[:,i]*(np.sqrt(1-rho**2)*deltaW1[:,i]+rho*deltaW2[:,i])
    mc = S[:,n]
    mc = np.maximum(mc-K,0)
    price = np.mean(mc)
    price_std = np.std(mc)/np.sqrt(m)
    lo = price - norm.ppf(0.975)*price_std
    hi = price + norm.ppf(0.975)*price_std
    return (price,lo,hi)
 

In [64]:
m = 2000
T = 1
F0 = 100
K = 100
sigma_0 = 0.04
cmcOptionPrice(T, m, F0, K,sigma_0 )



(8.898018991921496, 8.294691383236408, 9.501346600606585)

##Willard's Formula
Let us price a call option:
$$V_{T}=E\lbrack(S_{T}-K)_{+}\rbrack$$
Notice that
\begin{align*}
S_{T}&=S_{0}\exp\left(-\frac{1}{2}\int_{0}^{T}\sigma_{s}^{2}ds+\int_{0}^{T}\sigma_{s}(\sqrt{1-\rho^{2}}B_{s}+\rho W_{s})\right)\\&=S_{0}\exp\left(-\frac{1}{2}(\bar{\sigma_{p}^{2}}T+\rho^{2}\int_{0}^{T}\sigma_{s}^{2}ds)\right)\times\exp(\sqrt{1-\rho^{2}}\int_{0}^{T}\sigma_{s}dB_{s}+\rho\int_{0}^{T}\sigma_{s}dW_{s})
\end{align*}
where $\bar{\sigma_{\rho}}^{2}=\frac{1}{T}\left(\int_{0}^{T}\sigma_{s}^{2}ds-\rho^{2}\int_{0}^{T}\sigma_{s}^{2}ds\right)$.We assume that $\xi=\exp(\rho\int_{0}^{T}\sigma_{s}dW_{s}-\frac{1}{2}\rho^{2}\int_{0}^{T}\sigma_{s}^{2}ds)$. Then we can write $S_{T}$ as 
$$S_{T}=S_{0}\xi\exp(-\frac{1}{2}\bar{\sigma_{\rho}}^{2}T+\sqrt{1-\rho^{2}}\int_{0}^{T}\sigma_{s}dB_{s} )$$
Note that $\sqrt{1-\rho^{2}}\int_{0}^{T}\sigma_{s}dB_{s}=N(0,\bar{\sigma_{p}}^{2}T)$
$$S_{T}=S_{0}\xi\exp(-\frac{1}{2}\bar{\sigma_{p}}^{2}T+\bar{\sigma_{p}}\sqrt{T}N(0,1))$$
That is, $S_{T}$ can be seen (conditioned to $W$) as a process with deterministric volatility given by $\bar{\sigma_{\rho}}$ 
and the initial asset price is $S_{0}\xi$
By the Law of Iterated Expectations, we can re-write $V_{T}$ as 
\begin{align*}
V_{T}&=E\lbrack E\lbrack (S_{T}-K)_{+}|W_{t}\rbrack\rbrack\\
&=E\lbrack BS(S_{0}\xi,K,\bar{\sigma_{p}}^{2},T)\rbrack
\end{align*}


To implement the Monte Carlo simulation, the discretization for the volatility is the first step and given by 
$$\sigma_{t_{i}}^{2}=\max({\sigma_{t_{i-1}}^{2}}+\kappa(\theta-\sigma_{t_{i-1}}^{2}\Delta t)+\xi\sqrt{\sigma_{t_{i-1}}^{2}}\Delta W_{i-1},0)$$

We take $S_{0}=K=100,T=1$ and $\kappa=1,\theta=0.09, \sigma_{0}=0.2,\xi=0.3,\rho=-0.5,n=5000$ in the simulation. 



In [60]:
def BlackScholes(F0, K, sigma, T, r=0):
    d1 = (np.log(F0 / K) + sigma**2 / 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    BSprice = np.exp(-r * T) * (F0 *  ss.norm.cdf(d1) - K * ss.norm.cdf(d2))
    return(BSprice)

def mcOptionPrice(Z,T, m, F0, K,sigma_0,rho= -0.5, gamma=0.3, theta=0.09, ka=1,n=5000,r=0):
    OptionPrice = 0
    for i in range(0,m):
        Z1 = Z[i,]
        Z2 = Z1*np.sqrt(T/n)
        W2 = np.zeros(n+1)
        W2[1:n+1] = np.cumsum(Z2)
        df_W2 = np.diff(W2)
        sigma = np.zeros(n+1)
        sigma[0] = sigma_0
        for j in range(1,n+1):
            a = sigma[j-1]**2+ka*(theta-sigma[j-1]**2)*T/n + gamma * sigma[j-1]*df_W2[j-1]
            if (a>0): sigma[j] = np.sqrt(a)
            else: sigma[j] = 0
        # stochastic volatility compute from the GBM from 0 to T
        # integral approximation as finite sums
        deterministic_integral = np.sum(sigma[0:-1]**2)* T / n       # 1/n := constant time step, then 1/n = t(i+1)-t(i) for all i
        stochastic_integral    = np.dot(sigma[0:-1],np.diff(W2))   
        ksi_0 = np.exp(rho * stochastic_integral - 0.5 * rho**2 * deterministic_integral)    
        sigma_rho = np.sqrt(1/T * (1-rho**2) * deterministic_integral)
        OptionPrice += BlackScholes(F0 * ksi_0, K, sigma_rho, T)/m    
    return(OptionPrice)



In [61]:
 n = 5000
 m = 2000
 Z = np.random.normal(0,1,[m,n])
 T = 1
 F0 = 100
 K = 100
 sigma_0 = 0.2
 mcOptionPrice(Z,T, m, F0, K,sigma_0)

9.184149096155663